# Dropbox Connection

Use this template to load any data file stored on Dropbox into your workspace as explained in [this article](https://workspace-docs.datacamp.com/integrations/dropbox). Using the Integrations tab, you can make this connection without exposing sensitive data; you can learn more about integrations [here](https://workspace-docs.datacamp.com/integrations/environment-variables).

As an example, we have provided you with three sample csv files; you can find more info on these tables in the appendix of this template. To connect to the Dropbox folder that stores these files, you will have to set the right environment variables. You can find these in [this section](https://workspace-docs.datacamp.com/integrations/dropbox#sample-database-online-shoe-retail).

In [1]:
# Import the packages
import pandas as pd
import os
from io import StringIO
from dropbox import Dropbox

In [2]:
# create a Dropbox object using the access token
try:
    dbx = Dropbox(os.environ["DROPBOX_ACCESS_TOKEN"])
except:
    print("Token error")

In [3]:
# helper function to read files from Dropbox and load it into a dataframe
def read_file(filename, index_col = None):
    metadata, res = dbx.files_download(path="/"+filename)
    s=str(res.content,"utf-8")
    data = StringIO(s) 
    return pd.read_csv(data, index_col=index_col)

## Access your files using your access token

In [4]:
# Load the files into a pandas dataframe using the access token
customers = read_file("customers.csv", index_col = "CustomerId")
orders = read_file("orders.csv", index_col = "OrderId")
products = read_file("products.csv", index_col = "ItemId")

#### Example query
Let's say you would like to follow up on all the customers that have not received their order yet. So, you would need to extract their emails and order numbers.

In [5]:
emails_orders_not_received =( orders
                                    .query("Status != 'received'")
                                    .join(customers, on="CustomerId")
                            )["Email"]

emails_orders_not_received

OrderId
3    aagar3@washingtonpost.com
4            bwaumsley1@hp.com
6         rdarcey4@answers.com
7         roaker5@mashable.com
7         roaker5@mashable.com
Name: Email, dtype: object

## Appendix

#### customers.csv

| Column name | Description                    | Values  |
|:-------------|:--------------------------------|:---------|
| CustomerId  | key; unique id of the customer | integer |
| FirstName   | first name of the customer     | string  |
| LastName    | last name of the customer      | string  |
| Email       | (unique) email of the customer | string  |
| Address     | address of the customer        | string  |
| City        | city the customer lives in     | string  |
| Country     | country the customer lives in  | string  |

#### products.csv

| Column name | Description                                       | Values  |
|:-------------|:---------------------------------------------------|:---------|
| ItemId      | key; unique id of item                            | integer |
| ProductName | the retail name of the item (name - color - size) | string  |
| Price       | the price of the item (in USD)                    | float   |

#### orders.csv

| Column name | Description                                                 | Values  |
|-------------|:-------------------------------------------------------------|:---------|
| OrderId     | key; unique id of order                                     | integer |
| CustomerId  | foreign key; the unique id of the buying customer           | integer |
| ItemId      | foreign key; the unique id of the item bought               | integer |
| Amount      | amount bought of the item                                   | integer |
| DateTime    | date and time the item was bought (YYYY-MM-DD HH:mm:ss)     | string  |
| Status      | the status of the order (paid, shipped, received, returned) | string  |